In [32]:
pwd

'/home/wsuser/work'

In [33]:
!pip install keras
!pip install tensorflow

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [35]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [36]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AiZRS_D-r5y6Pgg_FB2No8ZsrHi3bu1VSOm0p0LK6DzV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'cnnecgvj-donotdelete-pr-4ralnc6ghus6ny'
object_key = 'data.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [37]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [38]:
pwd

'/home/wsuser/work'

In [39]:
import os
filenames = os.listdir('/home/wsuser/work/data/train')

In [40]:
x_train = train_datagen.flow_from_directory("/home/wsuser/work/data/train",target_size = (64,64),batch_size = 32,class_mode = "categorical")
x_test = test_datagen.flow_from_directory("/home/wsuser/work/data/test",target_size = (64,64),batch_size = 32,class_mode = "categorical")

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [41]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [42]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

In [43]:
#MODEL BUILDING
model = Sequential()

In [44]:
model.add(Conv2D(32,(3,3),input_shape = (64,64,3),activation = "relu"))

In [45]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [46]:
model.add(Conv2D(32,(3,3),activation='relu'))

In [47]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [48]:
model.add(Flatten()) # ANN Input...

In [49]:
model.add(Dense(32))
model.add(Dense(6, activation='softmax'))

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

In [51]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [52]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_164/53529210.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/10
480/480 [==============================] - 52s 107ms/step - loss: 1.0084 - accuracy: 0.6488 - val_loss: 0.6024 - val_accuracy: 0.7873
Epoch 2/10
480/480 [==============================] - 49s 103ms/step - loss: 0.2960 - accuracy: 0.9138 - val_loss: 0.4333 - val_accuracy: 0.8639
Epoch 3/10
480/480 [==============================] - 51s 106ms/step - loss: 0.2419 - accuracy: 0.9291 - val_loss: 0.4618 - val_accuracy: 0.8544
Epoch 4/10
480/480 [==============================] - 50s 104ms/step - loss: 0.2066 - accuracy: 0.9394 - val_loss: 0.4141 - val_accuracy: 0.8725
Epoch 5/10
480/480 [==============================] - 50s 105ms/step - loss: 0.1878 - accuracy: 0.9447 - val_loss: 0.3560 - val_accuracy: 0.9007
Epoch 6/10
480/480 [==============================] - 49s 102ms/step - loss: 0.1674 - accuracy: 0.9501 - val_loss: 0.3399 - val_accuracy: 0.9032
Epoch 7/10
480/480 [==============================] - 50s 105ms/step - loss: 0.1454 - accuracy: 0.9563 - val_loss: 0.3157 - val_ac

In [53]:
model.save('ECG.h5')

In [83]:
!tar -zcvf ECG-arrhythmia-classification-model_new.tgz ECG.h5

ECG.h5


In [84]:
ls -1

data/
ECG-arrhythmia-classification-model_new.tgz
ECG.h5
my_model.tar1.gz


In [56]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.6 MB/s eta 0:00:01


In [57]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"0FMoFudKPCRM351dg6DBvpLNZae1cjKCCPF_lbmrVuAi"
}
client=APIClient(wml_credentials)

In [58]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space ['resources'] if item['entity'] ["name"] == space_name)['metadata']['id'])

In [59]:
space_uid = guid_from_space_name( client,'CNN_ECG' )
print("Space UID = " + space_uid)

Space UID = 92524340-d705-4a82-b97d-1b201e39e889


In [60]:
client.set.default_space(space_uid)

'SUCCESS'

In [85]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [62]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [65]:
model_details = client.repository.store_model(model='ECG-arrhythmia-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN_ECG",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_id(model_details)

In [66]:
model_id

'328afef1-2c1f-4e75-9983-ff2c54af92ad'

In [67]:
client.repository.download(model_id,'my_model.tar1.gz')

Successfully saved model content to file: 'my_model.tar1.gz'


'/home/wsuser/work/my_model.tar1.gz'

In [68]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [69]:
model=load_model('ECG.h5')

In [74]:
pwd

'/home/wsuser/work'

In [76]:
ls

data/  ECG-arrhythmia-classification-model_new.tgz  ECG.h5  my_model.tar1.gz


In [ ]:

img=image.load_img(r'/home/wsuser/work/Unknown_image.png',target_size=(64,64))


In [ ]:
import numpy as ns
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [ ]:
pred = model.predict(x)


In [99]:
pred[0]

1


In [ ]:
index=['left Bundle Branch block','Normal','Premature Atrial Contraction','Premature Ventricular Contraction','Right Bundle Branch Block','Ventricular Fibrillation']


In [100]:
a = index[pred[0]]
print(a)

Normal
